# Homework 3
### Neil Kanungo - December 2022

Instructions: Have all the code for all the questions in one Jupyter notebook and upload the file at the time of submission. I will only consider this file format for grading. So, after you uploaded the file, check the file extension and if it is not an ipynb, then upload the right file.

Consider Titanic dataset and do the following:

1) Apply SVM along with GridSearchCV to determine the best parameters 
for kernel and C. Use the best parameters from the search and build an SVC model and compute confusion matrix, classification report and accuracy.

2) Likewise, use GridSearchCV to find the best number of trees for Random Forest and then build Random Forest Classifier and compute confusion matrix, classification report and accuracy.

Also read about RandomSearchCV.  And explain the difference between RandomSearchCV and GridSearchCV.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("Titanic.csv")

df

,PassengerId,Survived,Pclass,Name,Gender,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [2]:
keepCols = ['Survived','Pclass','Gender','Age','SibSp','Parch','Fare','Embarked']
df = df[keepCols]

df.isnull().sum()

Survived      0
Pclass        0
Gender        0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [3]:
df = df.dropna()

df.isnull().sum()

Survived    0
Pclass      0
Gender      0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [4]:
y = df['Survived']
x = df[['Pclass','Gender','Age','SibSp','Parch','Fare','Embarked']]

In [10]:
# Convert categorical columns to numeric
from sklearn.compose import make_column_selector as col_selector
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder, LabelEncoder

# categorical_cols_obj is a categorical object
categorical_cols_obj = col_selector(dtype_include=object)

# categorical_features is a list of  column names of features with catergorical data
categorical_features = categorical_cols_obj(df)

# numeric_features is a list of column names of features with numeric data
int_cols_obj = col_selector(dtype_include="int64")
numeric_features = int_cols_obj(df)

print("Categorical Features: " + str(categorical_features))
print("Numeric Features: " + str(numeric_features))

df_cat = df[categorical_features]
ob_encoded = OrdinalEncoder()
cat_encoded = ob_encoded.fit_transform(df_cat)

x = np.hstack([cat_encoded, np.array(df[numeric_features])])

Categorical Features: ['Gender', 'Embarked']
Numeric Features: ['Survived', 'Pclass', 'SibSp', 'Parch']


In [6]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20)

### SVM Classifier

In [7]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [8]:
parameters = {'kernel':['linear', 'rbf', 'poly'], "C": [0.1, 1, 10, 100], "gamma": [0.01, 0.1, 1, 10]}

svc = SVC()

clf = GridSearchCV(svc, param_grid= parameters, cv=3, verbose=True, n_jobs=1)

final_clf = clf.fit(x_train, y_train)

print("Best Parameters: " + str(final_clf.best_params_))

Fitting 3 folds for each of 48 candidates, totalling 144 fits
Best Parameters: {'C': 0.1, 'gamma': 0.01, 'kernel': 'linear'}


In [12]:
from sklearn import metrics

svc = SVC(kernel = 'linear', C=0.1, gamma=0.01) # use the best parameters
svc.fit(x_train, y_train) # fiting the data to the model

yc_pred = svc.predict(x_test)
print(confusion_matrix(y_test,yc_pred))
print(classification_report(y_test,yc_pred))
print("Accuracy:", metrics.accuracy_score(y_test, yc_pred))

[[88  0]
 [ 0 55]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        88
           1       1.00      1.00      1.00        55

    accuracy                           1.00       143
   macro avg       1.00      1.00      1.00       143
weighted avg       1.00      1.00      1.00       143

Accuracy: 1.0


### Random Forest Classifier

In [16]:
from sklearn.ensemble import RandomForestClassifier

# Create a Random Forest classifier
rf = RandomForestClassifier()

# GridSearch for best parameters
parameter = parameters = {'n_estimators':[100,200,300], "max_depth": [1,10,100], "min_samples_split": [2,4,6]}

clf = GridSearchCV(rf, param_grid= parameters, cv=3, verbose=True, n_jobs=1)

final_clf = clf.fit(x_train, y_train)

print("Best Parameters: " + str(final_clf.best_params_))


# Make predictions on the test set
y_pred = final_clf.predict(x_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best Parameters: {'max_depth': 1, 'min_samples_split': 6, 'n_estimators': 100}
[[88  0]
 [11 44]]
              precision    recall  f1-score   support

           0       0.89      1.00      0.94        88
           1       1.00      0.80      0.89        55

    accuracy                           0.92       143
   macro avg       0.94      0.90      0.92       143
weighted avg       0.93      0.92      0.92       143

Accuracy: 0.9230769230769231


### RandomSearchCV versus GridSearchCV

RandomSearchCV and GridSearchCV are both used for hyperparameter tuning using cross-validation to evaluate sets of given input hyperparameters for a model.

The main difference in the two is how they search for the best hyperparameters.

GridSearchCV will try all possible combinations of parameters given to it as a grid. RandomSearchCV, however, uses a random search of the given hyperparameters. In this sense, it does not test all possible combinations, however, it is likely to perform faster.